<a href="https://colab.research.google.com/github/RRDajay/Machine_Learning_Collection/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Hyperparameters
batch_size = 1000
num_epochs = 25

# Load data
transform = transforms.Compose([
                                transforms.ToTensor()
    ])

train_set = torchvision.datasets.CIFAR10(root='dataset/', train=True, transform=transform, download=True)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_set = torchvision.datasets.CIFAR10(root='dataset/', train=False, transform=transform, download=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
class Identity(nn.Module):

  def __init__(self):
    super().__init__()

  def forward(self, x):
    return x

# GPU or CPU selection
device = 'cuda' if torch.cuda.is_available else 'cpu'

# Model Creation

model = torchvision.models.vgg16(pretrained=True, progress=True)
# Freeze model
for param in model.parameters():
  param.requires_grad = False

# Create new linear layer, remove avgpool layer
model.avgpool = Identity()
model.classifier = nn.Sequential(
    nn.Dropout(),
    nn.BatchNorm1d(num_features=512),
    nn.Linear(512, 10)
    )

model = model.to(device)

print(model)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
# Optimizer and Loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Scheduler
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

In [31]:
# Model training 
n_total_steps =len(train_loader)
for epoch in range(num_epochs):

  losses = []

  for batch_idx, (data, targets) in enumerate(train_loader):
    data, targets = data.to(device), targets.to(device)

    # Forward
    scores = model(data)
    loss = criterion(scores, targets)
    losses.append(loss.item())

    # Backward
    optimizer.zero_grad()
    loss.backward()

    # Grad Descent
    optimizer.step()
    losses.append(loss.item())
    if (batch_idx) % 25 == 0:
      print(f"Epoch: {epoch+1}/{num_epochs}\t\tStep: {batch_idx+1}/{n_total_steps}\t\tloss: {sum(losses)/len(losses)}")
  
  scheduler.step(sum(losses)/len(losses)) 
  


Epoch: 1/25		Step: 1/50		loss: 1.3982295989990234
Epoch: 1/25		Step: 26/50		loss: 1.3492299685111413
Epoch    30: reducing learning rate of group 0 to 1.0000e-04.
Epoch: 2/25		Step: 1/50		loss: 1.3563973903656006
Epoch: 2/25		Step: 26/50		loss: 1.357703447341919
Epoch: 3/25		Step: 1/50		loss: 1.3689690828323364
Epoch: 3/25		Step: 26/50		loss: 1.3506311361606305
Epoch: 4/25		Step: 1/50		loss: 1.3469504117965698
Epoch: 4/25		Step: 26/50		loss: 1.3491741327139049
Epoch: 5/25		Step: 1/50		loss: 1.2829779386520386
Epoch: 5/25		Step: 26/50		loss: 1.3479869136443505
Epoch: 6/25		Step: 1/50		loss: 1.3154040575027466
Epoch: 6/25		Step: 26/50		loss: 1.3434474835029016
Epoch: 7/25		Step: 1/50		loss: 1.338165283203125
Epoch: 7/25		Step: 26/50		loss: 1.352046618094811
Epoch    36: reducing learning rate of group 0 to 1.0000e-05.
Epoch: 8/25		Step: 1/50		loss: 1.334945797920227
Epoch: 8/25		Step: 26/50		loss: 1.3386414463703449
Epoch: 9/25		Step: 1/50		loss: 1.3962491750717163
Epoch: 9/25		Step: 26/

In [1]:
# Model Validation
with torch.no_grad():

  n_correct, n_samples = 0, 0

  for data, labels in test_loader:
    
    data = data.to(device)
    labels = labels.to(device)
    
    outputs = model(data)
    _, predictions = torch.max(outputs, 1)
    n_samples += labels.shape[0]
    n_correct += (predictions == labels).sum().item()

  acc = 100 * n_correct/n_samples
  print(f"Acc: {acc}")


NameError: ignored

In [ ]:
# TODO:
# Apply normalization to image dataset
# Apply augmentation to image dataset
# Tensorboard 
# Improve validation code
